<a href="https://colab.research.google.com/github/Kennedy87670/NLP/blob/main/Introduction_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# introduction to NLP Fundamentals in Tensorflow

NLP has the goal of deriving information out of natural language (could be sequences text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq).

# Check our GPU

In [1]:
!nvidia-smi - l

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

#import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2022-08-26 13:41:04--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-08-26 13:41:04 (88.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we are going to be using is Kaggle's introduction to NLP dataset (text samples of tweets labeled as diaster or not diaster).

See the original source here https://www.kaggle.com/competitions/nlp-getting-started

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#unzip data
unzip_data('nlp_getting_started.zip')

--2022-08-26 13:41:06--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.45.112, 142.250.81.208, 142.251.163.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.45.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2022-08-26 13:41:06 (117 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



# Visualize a text dataset


In [4]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df =pd.read_csv('test.csv')

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
# what does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
#how many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
# how many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [9]:
# lets visualize some random samples
import random
random_index = random.randint(0, len(train_df)-5) #create random indexes
for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target:{target}", '(real diaster)' if target >0 else '(not real diaster' )
  print(f'Text:\n{text}\n')
  print('---\n')

Target:1 (real diaster)
Text:
@gilmanrocks7 wow. Where is this? For the rest of the summer I usually have to worry about forest fires. Calgary is about 8 hours away.

---

Target:0 (not real diaster
Text:
US wants future first responders to be more high-tech - http://t.co/1VI2RNbK2i

---

Target:0 (not real diaster
Text:
Quality Metrics Penalties May Harm Patient Care PCPs Say: Primary care physicians generally hold positive vi... http://t.co/TdwprVB04y

---

Target:0 (not real diaster
Text:
RT RoadID: Thanks to Alex for his story &amp; to all first responders for being there when we need you. Û_ http://t.co/HikDC1fM2F

---

Target:1 (real diaster)
Text:
Is it bad to say that I'm kinda afraid of storms and we are in a storm???? help

---



In [10]:
## split data into training and validation dataset
from sklearn.model_selection import train_test_split

In [11]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42)

In [12]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)


(6851, 6851, 762, 762)

In [13]:
# check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers
 
when dealing with text problem, one of the first things you will have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenziation - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of feature vecto for each token (the size of the feature vetor can be defined and this embedding can be learned)

## Text Vectorization (tokenization)

In [14]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None


In [16]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [17]:
# setup text vectorization variable
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length= 15  # max length our sequences will be (e.g how many words from a tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [18]:
# fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)


In [19]:
# create a sample sentence and tokenize it
sample_sentence = "Theres a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [20]:
# choose a random sentence from the training datset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"original text: \n {random_sentence}\
\n\nVectorized version: ")
text_vectorizer([random_sentence])

original text: 
 @troylercraft YEAH ITS NOT WORTH IT BC HE ALREADY HAS SO MANY SPAMMERS &amp; HIS TWITTER IS PROBABLY BLOWN UP EVERY SECOND

Vectorized version: 


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[7343,  663,   37,   34, 1343,   15,  632,   56,  659,   41,   28,
         123, 8253,   35,   79]])>

In [21]:
# get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()  #get all the unique words in our training data 
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"NMumber of words in vocab: {len(words_in_vocab)}")
print(f" 5 most common words:{top_5_words} ")
print(f" 5 most common words:{bottom_5_words} ")

NMumber of words in vocab: 10000
 5 most common words:['', '[UNK]', 'the', 'a', 'in'] 
 5 most common words:['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'] 


## Creatign an Embedding using Embedding layer

To make our embedding, we are going to use Tensorflow's embedding layers: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about for our embedding layer:
* Input dim = the size of our get_vocabulary
* output dim = the size of the output embedding vector, for example, a value of 100 would mean token get represented by a vector 100 long
* input_length = length of sequences passed to the embedding layers

In [22]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             embeddings_initializer='uniform',
                             output_dim=128,
                             input_length= max_length #how long each input
                             )
embedding

In [23]:
# Get a random sentence from the training set
random_sentence =random.choice(train_sentences)
print(f"Original text:\n {random_sentence} ")

# embedd the random sentence (turn it into dense vectors of fixed size)
sample_embed= embedding(text_vectorizer([random_sentence]))

sample_embed

Original text:
 Help yourself or those you love who suffer from self-esteem wounds. You can today! http://t.co/tu6ScRSXVG http://t.co/iDhj4JBQ05 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01453419, -0.02102694, -0.04275301, ..., -0.04279482,
          0.03921654,  0.01367651],
        [-0.02648089,  0.0419044 ,  0.00602508, ..., -0.00366461,
         -0.0331851 , -0.03981849],
        [ 0.01337525, -0.0327874 , -0.0098605 , ...,  0.01282879,
          0.03847585, -0.01986154],
        ...,
        [-0.02737098, -0.004019  , -0.04582789, ...,  0.04071346,
         -0.00492287,  0.02339828],
        [ 0.00137956, -0.03593028,  0.02241114, ..., -0.00532616,
         -0.04612503, -0.01560142],
        [-0.03319392, -0.01618034, -0.00771743, ..., -0.01209273,
          0.03735695, -0.01304443]]], dtype=float32)>

In [24]:
# Check out a single token embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.01453419, -0.02102694, -0.04275301, -0.01460357, -0.00756574,
        -0.03766122, -0.04422035,  0.01092928,  0.0264936 , -0.01920066,
         0.00208674, -0.02100723,  0.01978922, -0.01300158, -0.02548047,
        -0.01213151,  0.02252575, -0.01789586, -0.01285381,  0.0112384 ,
         0.03295777,  0.04032629, -0.03408279, -0.00394513, -0.00473825,
        -0.01965061,  0.03245715,  0.03431242,  0.03588295,  0.00660434,
        -0.03006301, -0.04936831, -0.00723041, -0.01842844, -0.04411573,
        -0.01125473,  0.0298975 ,  0.03468349, -0.00674039, -0.03318732,
        -0.03212549, -0.03967072, -0.03196301,  0.03291072,  0.01379769,
         0.0304472 ,  0.00042354, -0.00101414,  0.01947126,  0.04335375,
         0.03445489, -0.00331949,  0.04929333, -0.04280538,  0.03584   ,
         0.04175666,  0.0034937 , -0.02360053,  0.00983542, -0.03981607,
         0.01299297, -0.00433994,  0.01260031, -0.00338858,  0.00891972,
  

# Modelling a text dataset (running a series of experiments)

Now we have got a way tot turn our text sequences into numbers, its time to start buildinjg a series of modelling experiments.

We will start witha  baseline and move on from there.

* Model 0: naive bayes (baseline), this is from sklearn ML map
* Model 1: Feed-forward neaural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* model 5: 1D convolutional Neautral network (CNN)
* model 6: Tensorflow hub Pretrained Feature Extractor (using transfer learning for NLP)
* mODEL 7: SAME AS MODEL 6 WITH 10% OF TRAINING DAta

how are we going to approach all these?

use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Fit a model
* Evaluate our modelm

## model 0 : getting a baseline

As with all machine learning modelling experiments, its important to create a baseline model so you have got a benchmark for future experiments to build upon.
To create a baseline, we will use sklearn multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers

🔯 Note: its common pratice to use non DL algorithms as baseline of their speed and then later using DL to see if you can improve upon them.

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# create a tokenization and modelling pipeline
model_0 = Pipeline([
                    ('tfidf', TfidfVectorizer()),  #convert words to numbers using tfid
                    ('clf', MultinomialNB()) # model the text
    
])

# fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [26]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

our baseline model achieves an accuracy of: 79.27%


In [27]:
# make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

Creating an evaluation function for our model experiments
We could evaluate these as they are but since we're going to be evaluating several models in the same way going forward, let's create a helper function which takes an array of predictions and ground truth labels and computes the following:

* Accuracy
* Precision
* Recall
* F1-score

🔑 Note: Since we're dealing with a classification problem, the above metrics are the most appropriate. If we were working with a regression problem, other metrics such as MAE (mean absolute error) would be a better choice.

In [28]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [29]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 1: A simple dense model
The first "deep" model we're going to build is a single layer dense model. In fact, it's barely going to have a single layer.

It'll take our text and labels as input, tokenize the text, create an embedding, find the average of the embedding (using Global Average Pooling) and then pass the average through a fully connected layer with one output unit and a sigmoid activation function.

If the previous sentence sounds like a mouthful, it'll make sense when we code it out (remember, if in doubt, code it out).

And since we're going to be building a number of TensorFlow deep learning models, we'll import our create_tensorboard_callback() function from helper_functions.py to keep track of the results of each.

In [30]:
# Create a tensorboard call back (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# create a directory to save TensorBoard logs
SAVE_DIR = 'model_logs'

In [31]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model


In [32]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [33]:
# Compile model
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [34]:
# Fit the model
model_1_history = model_1.fit(x= train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              y= train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20220826-134108
Epoch 1/5
215/215 [==============================] - 3s 11ms/step - loss: 0.6115 - accuracy: 0.6885 - val_loss: 0.5341 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.4399 - accuracy: 0.8206 - val_loss: 0.4699 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3464 - accuracy: 0.8587 - val_loss: 0.4567 - val_accuracy: 0.7953
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.2839 - accuracy: 0.8892 - val_loss: 0.4648 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.2368 - accuracy: 0.9124 - val_loss: 0.4790 - val_accuracy: 0.7822


In [35]:
# check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4790 - accuracy: 0.7822


[0.4789671003818512, 0.7821522355079651]

In [36]:
# maqke some predictions and evaluate those
model_1_pred_probs= model_1.predict(val_sentences)
model_1_pred_probs.shape

(762, 1)

In [37]:
# look at a single preiction
model_1_pred_probs[0]

array([0.34201497], dtype=float32)

In [38]:
# look at thr first 10 preiction
model_1_pred_probs[:10]

array([[0.34201497],
       [0.72711927],
       [0.9977863 ],
       [0.14399847],
       [0.10920942],
       [0.9370699 ],
       [0.87579006],
       [0.9939775 ],
       [0.96433043],
       [0.33163035]], dtype=float32)

In [39]:
# convert model predictions probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [40]:
#calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.21522309711287,
 'precision': 0.7864332425219001,
 'recall': 0.7821522309711286,
 'f1': 0.7792361147360404}

In [41]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [42]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## Visualize Embedding

In [43]:
# Get the vocabulary from the text vectorization layers
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [44]:
# model_1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [45]:
# Get the weight matrix of embedding layer
# (these are the numerical representations of each token in our training data, which have been learnt by 5 epochs)
embed_weights= model_1.get_layer('embedding').get_weights()[0]
print(embed_weights.shape) # same size as vocab size and embedding dim (output dim of our embedding layer)

(10000, 128)


Now we've got the embedding matrix model has learned to represnt our tokens, lets see how we can visualize it.

To do so. Tensorflow has a handy tool called projector:
https://www.tensorflow.org/text/guide/word_embeddings

In [46]:
# create embedding files (we got this from tensorflow's word embedding documentation)
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()


In [47]:
# Download files from colab to upload to projector
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

## Recurrent Neural Network (RNN'S)
RNN's are useful for sequence data

The premise of a recurrent neural network is to use the represntation of a later night

## Model 2: LSTM

LSTM = long short term memory (one of the most popular LSTM cells)
Our structure of an RNN typically looks like this:

inputs(text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output (label probability)

In [48]:
# Create an LSTM mOdel
from tensorflow.keras import layers


# Create LSTM model
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = layers.LSTM(64)(x) # return vector for whole sequence
print(x.shape)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")


(None, 15, 128)
(None, 64)


In [49]:
# get a summary()
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [50]:

# compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [51]:
# Fit model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "LSTM")])


Saving TensorBoard log files to: model_logs/LSTM/20220826-134122
Epoch 1/5
215/215 [==============================] - 7s 19ms/step - loss: 0.2225 - accuracy: 0.9226 - val_loss: 0.5434 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1559 - accuracy: 0.9438 - val_loss: 0.7462 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1269 - accuracy: 0.9512 - val_loss: 0.6956 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.1030 - accuracy: 0.9591 - val_loss: 0.7387 - val_accuracy: 0.7808
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0825 - accuracy: 0.9672 - val_loss: 0.7882 - val_accuracy: 0.7756


In [52]:
# make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[8.9303464e-02],
       [8.1441545e-01],
       [9.9950725e-01],
       [1.5341523e-01],
       [5.9020519e-04],
       [9.8962831e-01],
       [6.8929720e-01],
       [9.9966824e-01],
       [9.9917305e-01],
       [4.0424305e-01]], dtype=float32)

In [53]:

# convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [54]:

val_labels

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In [55]:
# calculate model 2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results


{'accuracy': 77.55905511811024,
 'precision': 0.7769617885542762,
 'recall': 0.7755905511811023,
 'f1': 0.7737412623092624}

## Model 3: GRU

Another popular and effective RNN component is the GRU or gated recurrent unit.

The GRU cell has similar features  to an LSTM cell but has less parameters

In [56]:
from IPython.core.inputtransformer2 import leading_empty_lines
# build an RNN using GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
x = layers.GRU(64, return_sequences=True)(x) # if you want to stack recurrent layers on top of each other, you need return sequence true
# print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x)
# print(x.shape)
# x = layers.GRU(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation='relu')(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name='model_3_GRU')

In [57]:
# Get a summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 15, 64)            37248     
                                                                 
 global_average_pooling1d_1   (None, 64)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 1)                 

In [58]:
# Compile GRU model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [59]:
# Fit model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "GRU")])

Saving TensorBoard log files to: model_logs/GRU/20220826-134146
Epoch 1/5
215/215 [==============================] - 4s 13ms/step - loss: 0.1784 - accuracy: 0.9501 - val_loss: 0.6563 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.1021 - accuracy: 0.9622 - val_loss: 0.7080 - val_accuracy: 0.7730
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0844 - accuracy: 0.9679 - val_loss: 0.8526 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0732 - accuracy: 0.9707 - val_loss: 0.9965 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0608 - accuracy: 0.9749 - val_loss: 1.2432 - val_accuracy: 0.7625


In [60]:
# Make predictions on the validation data
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

((762, 1), array([[3.7513971e-03],
        [9.4398403e-01],
        [9.9996990e-01],
        [6.1544448e-02],
        [5.3244829e-04],
        [9.9996662e-01],
        [9.9525809e-01],
        [9.9999607e-01],
        [9.9998677e-01],
        [7.1543115e-01]], dtype=float32))

In [61]:

# Make predictions on the validation data
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

((762, 1), array([[3.7513971e-03],
        [9.4398403e-01],
        [9.9996990e-01],
        [6.1544448e-02],
        [5.3244829e-04],
        [9.9996662e-01],
        [9.9525809e-01],
        [9.9999607e-01],
        [9.9998677e-01],
        [7.1543115e-01]], dtype=float32))

In [62]:
# Convert prediction probabilities to prediction classes
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [63]:
# Calcuate model_3 results
model_3_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 76.24671916010499,
 'precision': 0.7628028463824278,
 'recall': 0.7624671916010499,
 'f1': 0.7609952154742672}

## Model_4 : bidirectional RNN

nORMALRNN'S GO FROM LEFT TO RIGHT (JUST LIKE YOU READ AN ENGLISH sentence) however, a bidirectional RNN goes from right to left as well as left to right.


In [64]:
# build a bidirection RNN in Tensorflow
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.Bidirectional(layers.LSTM(64, return_sequences=True)) (x)
#print(x.shape)
x = layers.Bidirectional(layers.LSTM(64)) (x)
outputs= layers.Dense(1, activation="sigmoid") (x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

In [65]:
# get a summary
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [66]:
# Compile
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [67]:
# Fit the model (takes longer because of the bidirectional layers)
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "bidirectional_RNN")])


Saving TensorBoard log files to: model_logs/bidirectional_RNN/20220826-134209
Epoch 1/5
215/215 [==============================] - 7s 21ms/step - loss: 0.1101 - accuracy: 0.9656 - val_loss: 0.9406 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0584 - accuracy: 0.9761 - val_loss: 1.2315 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0454 - accuracy: 0.9781 - val_loss: 1.1992 - val_accuracy: 0.7598
Epoch 4/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0419 - accuracy: 0.9812 - val_loss: 1.3248 - val_accuracy: 0.7598
Epoch 5/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0444 - accuracy: 0.9804 - val_loss: 1.3945 - val_accuracy: 0.7638


In [68]:
# Make predictions with bidirectional RNN on the validation data
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[2.0715088e-02],
       [8.9863527e-01],
       [9.9997807e-01],
       [2.2072113e-01],
       [1.3396144e-04],
       [9.9992669e-01],
       [9.9667978e-01],
       [9.9998438e-01],
       [9.9997127e-01],
       [9.9913418e-01]], dtype=float32)

In [69]:
# Convert prediction probabilities to labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [70]:
# Calculate bidirectional RNN model results
model_4_results = calculate_results(val_labels, model_4_preds)
model_4_results

{'accuracy': 76.37795275590551,
 'precision': 0.763377762834769,
 'recall': 0.7637795275590551,
 'f1': 0.7633188054110939}

### Convolutional Neaural networks for text (and other types of sequences)

We've used CNNs for images but images are typically 2d (height x width)... however, our text data is 1D.

Previously we've Conv2D for our images but now we're going to use Conv1D.

The typical structure of a Conv1D model for sequences (in our case, text):

Inputs (text) -> Tokenization -> Embedding -> Layers(s) (typically Conv1D + pooling) -> Outputs (class probabilities)

# model_5 Conv1D

In [71]:
# testing out our embedding layer, Conv1D layer and max pooling
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn target sequence into ebedding
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5, # ngram of 5, reads 5 words at atime
                        activation="relu",
                        padding="valid") # default = "valid" the output is smaller
conv_1d_output = conv_1d(embedding_test) #pass test embedding through conv1d layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # equivalent to get the4 most feature or get the feature with the higest

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [72]:
# create 1-dimentional convolutional layer to model sequences
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

# Compile Conv1D model
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary of our 1D convolution model
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 32)            20512     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 1)              

In [73]:
# Fit the model
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "Conv1D")])


Saving TensorBoard log files to: model_logs/Conv1D/20220826-134253
Epoch 1/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1304 - accuracy: 0.9561 - val_loss: 0.8364 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0741 - accuracy: 0.9734 - val_loss: 1.0098 - val_accuracy: 0.7664
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0598 - accuracy: 0.9769 - val_loss: 1.1050 - val_accuracy: 0.7572
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0539 - accuracy: 0.9777 - val_loss: 1.1560 - val_accuracy: 0.7612
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0517 - accuracy: 0.9775 - val_loss: 1.1786 - val_accuracy: 0.7598


In [74]:
# Make predictions with model_5
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

array([[1.16224825e-01],
       [9.25098360e-01],
       [9.99902725e-01],
       [5.54217994e-02],
       [2.99825757e-07],
       [9.95444596e-01],
       [9.68194783e-01],
       [9.99935508e-01],
       [9.99999642e-01],
       [9.10075188e-01]], dtype=float32)

In [75]:
# Convert model_5 prediction probabilities to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [76]:
# Calculate model_5 evaluation metrics 
model_5_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_5_preds)
model_5_results

{'accuracy': 75.98425196850394,
 'precision': 0.7596690600590813,
 'recall': 0.7598425196850394,
 'f1': 0.7587882527192952}

## Model 6: Tensorflow hub pretarined sentence encoder Transfer learning

In [77]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([ sample_sentence,
    "When you can the universal sentence encoder on a sentence,it",])

print(embed_samples[0][:50])

tf.Tensor(
[-0.01157025  0.02485911  0.02878051 -0.012715    0.03971541  0.08827761
  0.02680988  0.05589838 -0.01068731 -0.00597293  0.00639321 -0.01819516
  0.00030816  0.09105889  0.05874645 -0.03180629  0.01512474 -0.05162925
  0.00991366 -0.06865345 -0.04209306  0.0267898   0.03011009  0.00321065
 -0.00337968 -0.04787356  0.0226672  -0.00985927 -0.04063615 -0.01292093
 -0.04666382  0.05630299 -0.03949255  0.00517682  0.02495827 -0.07014439
  0.0287151   0.0494768  -0.00633978 -0.08960193  0.02807119 -0.00808364
 -0.01360601  0.05998649 -0.10361788 -0.05195372  0.00232958 -0.02332531
 -0.03758106  0.03327729], shape=(50,), dtype=float32)


In [78]:
# Create a keras layer using the USE pretrained layer from tensorflow hub
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [104]:
# Create model using Sequential API
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation="sigmoid")
], name= "model_6_USE")


# Compile model
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_6.summary()



Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_11 (Dense)            (None, 128)               65664     
                                                                 
 dense_12 (Dense)            (None, 64)                8256      
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,871,809
Trainable params: 73,985
Non-trainable params: 256,797,824
_________________________________________________________________


In [105]:
# Train a classifier on top of pretrained embeddings
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "tf_hub_sentence_encoder")])


Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20220826-140030
Epoch 1/5
215/215 [==============================] - 3s 8ms/step - loss: 0.4726 - accuracy: 0.7905 - val_loss: 0.4280 - val_accuracy: 0.8110
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3954 - accuracy: 0.8286 - val_loss: 0.4280 - val_accuracy: 0.8136
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3631 - accuracy: 0.8441 - val_loss: 0.4357 - val_accuracy: 0.8189
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3288 - accuracy: 0.8619 - val_loss: 0.4193 - val_accuracy: 0.8228
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2882 - accuracy: 0.8847 - val_loss: 0.4403 - val_accuracy: 0.8281


In [106]:
# Make predictions with USE TF Hub model
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

array([[0.10068798],
       [0.88940746],
       [0.9993521 ],
       [0.1678355 ],
       [0.8602805 ],
       [0.8798632 ],
       [0.9891447 ],
       [0.9974272 ],
       [0.9871218 ],
       [0.05944699]], dtype=float32)

In [107]:
# Convert prediction probabilities to labels
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [108]:
# Calculate model 6 performance metrics
model_6_results = calculate_results(val_labels, model_6_preds)
model_6_results

{'accuracy': 82.80839895013123,
 'precision': 0.8313853191294134,
 'recall': 0.8280839895013123,
 'f1': 0.826476160433721}

In [108]:
## Model 7: TF Hub pretrained USE but with 10% of training data